## Writing this program in ipython notebook, since it is easy to visualize the results and stop after a step with data in memory..

### Importing necessary modules

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import pandas as pd
from io import StringIO

## URL for indian medical register

In [ ]:
url = "https://www.nmc.org.in/information-desk/indian-medical-register/"

### Starting the driver

In [ ]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(url)
driver.maximize_window()

### Getting the table containing doctors name and data

In [ ]:
#### Selecting Year

In [ ]:
time.sleep(5)  # time to load the page
select_year = driver.find_element(By.ID, "doctorYear")
select = Select(select_year)
year = driver.find_element(By.XPATH, '//*[@id="doctorYear"]/option[4]')
driver.execute_script('arguments[0].setAttribute("style","display:block;");', select_year)
select.select_by_visible_text('2020')

#### Selecting state

In [ ]:
# Selecting state
select_state = driver.find_element(By.ID, "advsmcId")
select = Select(select_state)
bihar_state = driver.find_element(By.XPATH, '//*[@id="advsmcId"]/option[6]')
driver.execute_script('arguments[0].setAttribute("style","display:block;");', select_state)
select.select_by_value('4')

#### Clicking Submit button to get_data

In [ ]:
submit = driver.find_element(By.ID, "doctor_advance_Details")
submit.click()
time.sleep(5)

## Get the data

In [ ]:
all_doctors = []
while True:
    # Get current status of webpage
    webpage = driver.page_source
    # Wrapping in StringIO because direct parsing is depreciated
    df = pd.read_html(StringIO(webpage))
    all_doctors.append(df[-1])
    
    # Finding and clicking next button if needed
    next_button = driver.find_element(By.ID, "doct_info5_next")
    classes = next_button.get_attribute('class')
    if 'disabled' in classes:
        break
    else:
        next_button.click()
        time.sleep(5)

## Quitting driver

In [ ]:
driver.quit()

# Combining all data, and saving it to csv

In [ ]:
# combining all_doctors
all_doctors_df = pd.concat(all_doctors, axis=0)
all_doctors_df.to_csv('all_doctors.csv', index=False, columns=['Sl. No.', 'Year of Info', 'Registration Number', 'State Medical Councils', 'Name', 'Father Name'])